In [1]:
import os
import json
import requests

import matplotlib.pyplot as plt
import seaborn as sns

from collections import defaultdict

In [2]:
RIGpath = "../data/RIG/nogaps/"
WUSSpath = "../data/SS_cons/SS_cons_WUSS.tsv"

bear90path = RIGpath + "bear_90_RIGs.tsv"
bear50path = RIGpath + "bear_50_RIGs.tsv"
qbear90path = RIGpath + "qbear_90_RIGs.tsv"
qbear50path = RIGpath + "qbear_50_RIGs.tsv"
zbear90path = RIGpath + "zbear_90_RIGs.tsv"
zbear50path = RIGpath + "zbear_90_RIGs.tsv"

In [ ]:

# RF from consensus list and request primary sequence alignments from RFAM
def fromTextToAlign(rtext):
    """converts a fast response from the API into a string array, ignoring IDs
    input:
        - r.text from response (format: http://rfam.org/family/{RF}/alignment/fasta)
    output:
        - String array with alignments
    """
    al = []
    for a in r.text.split(">"):
        if a:
            al.append("".join(a.split("\n")[1:]))
    return al

#read WUSS dictionary
families = defaultdict(dict)

with open(WUSSpath) as f:
    for line in f:
        #RF\tWUSS
        dat = line.split()
        print(dat[0])
        families[dat[0]]['SS_cons'] = dat[1].strip()
        r = requests.get('http://rfam.org/family/{}/alignment/fasta'.format(dat[0]))
        families[dat[0]]['align'] = fromTextToAlign(r.text)


RF00001
RF00002
RF00003
RF00004
RF00005
RF00006
RF00007
RF00008
RF00009
RF00010
RF00011
RF00012
RF00013
RF00014
RF00015
RF00016
RF00017
RF00018
RF00019
RF00020
RF00021
RF00022
RF00023
RF00024
RF00025
RF00026
RF00027
RF00028
RF00029
RF00030
RF00031
RF00032
RF00033
RF00034
RF00035
RF00036
RF00037
RF00038
RF00039
RF00040
RF00041
RF00042
RF00043
RF00044
RF00045
RF00046
RF00047
RF00048
RF00049
RF00050
RF00051
RF00052
RF00053
RF00054
RF00055
RF00056
RF00057
RF00058
RF00059
RF00060
RF00061
RF00062
RF00063
RF00064
RF00065
RF00066
RF00067
RF00068
RF00069
RF00070
RF00071
RF00072
RF00073
RF00074
RF00075
RF00076
RF00077
RF00078
RF00079
RF00080
RF00081
RF00082
RF00083
RF00084
RF00085
RF00086
RF00087
RF00088
RF00089
RF00090
RF00091
RF00092
RF00093
RF00094
RF00095
RF00096
RF00097
RF00099
RF00100
RF00101
RF00102
RF00103
RF00104
RF00105
RF00106
RF00107
RF00108
RF00109
RF00110
RF00111
RF00112
RF00113
RF00114
RF00115
RF00116
RF00117
RF00118
RF00119
RF00121
RF00122
RF00124
RF00125
RF00126
RF00127
RF00128


In [ ]:
#2 remove gaps by looking at the consensus
def removeGapsFromAlign(sscons, align, gapCharacter = "."):
    """
    removes positions from the alignment that match gaps in the sscons
    input: 
        sscons in WUSS format
        alignment as string array
    output:
        modified alignment 
    """
    assert len(sscons) == len(align[0]), "different lengths!\n sscons w gaps:{}, align:{}".format(
                                                    (
                                                    len(sscons)),
                                                    len(align[0])
                                                    )
    cleanAlign = ["".join([a for ss,a in zip(sscons,seq) if ss != gapCharacter]) for seq in align]
    if len(sscons) - sscons.count(gapCharacter) != len(cleanAlign[0]):
        raise ValueError('gaps removal error!')
    return cleanAlign

print('test clean align')
print(removeGapsFromAlign("aaaa.....aaaa", ["AAAABBBBBAAAA","AAAABBBBBAAAA","BBBBAAAAABBBB"]))


for rf_ in families:
    print(rf_)
    errors=[]
    try:
        families.get(rf_)['gapless'] = removeGapsFromAlign(families.get(rf_).get('SS_cons'), families.get(rf_).get('align'))
    except AssertionError:
        errors.append(rf_)

In [95]:
#3 per ogni posizione dell'allineamento calcolare l'entropia di shannon (solo su PFM)
import numpy as np
def alignmentToShannon(align):
    """
    given an array of strings, compute the per position shannon entropy
    input:
        - array of strings (same length)
    output:
        - array of shannon entropies
    """
    if align is None:
        return []
    shannon = []
    for pos in range(len(align[0])):
        column_string = "".join([seq[pos] for seq in align])
        H = seq_RIG_H(column_string)
        shannon.append(H)
    return shannon
from collections import Counter
def seq_RIG_H(s, char_amount=4):
    """log2 entropy of a string (excluding gaps)
    char_amount is used to get the normalized RIG (4 if standard nt)
    """
    gap_proportion = s.count("-")/len(s)
    s=s.replace("-","")
    probabilities = [n_x/len(s) for x,n_x in Counter(s).items() ]
    e_x = [-p_x*np.log2(p_x) for p_x in probabilities]   
    
    ##gap riducono in proporzione lo score finale 
    ##(as per https://onlinelibrary.wiley.com/doi/full/10.1002/prot.10146)
    return (np.log2(char_amount) - sum(e_x))/np.log2(char_amount) * (1-gap_proportion)


print('test seq_RIG_H')
print(seq_RIG_H('aaaaaaaab'))

print('test alignmentToShannon')
print(alignmentToShannon([
  'GAAACGGAGCGGCACCUCUUUUAACCCUUGAAGUCACUGCCCGUUUCGAGAGUUUCUC---AACUCGAA-UAACUAAAGCCAACGUGAACUUUUGCGGAUCUCCAGGAUCC---',
  'GAAACGGAGCGGCACCUCUUUUAACCCUUGAAGUCACUGCCCGUUUCGAGAGUUUCUC---AACUCGAA-UAACUAAAGCCAACGUGAACUUUUGCGGAUCUCCAGGAUCC---',
  'GAAACGGAGCGGCACCUCUUUUAACCCUUGAAGUCACUGCCCGUUUCGAGAGUUUCUC---AACUCGAA-UAACUAAAGCCAACGUGAACUUUUGCGGAUCUCCAGGAUCCGCU',
  'AGAACGGAGCGGUUUCUCGUUUAACCCUUGAAGACACCGCCCGUUCAGAGGGUAUCUCUCGAACCCGAAAUAACUAAAGCCAACGUGAACUUUUGCGGACCUC--UGGUCCGCU']))

test seq_RIG_H
0.7483708326121772
test alignmentToShannon
[0.5943609377704335, 0.5943609377704335, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5943609377704335, 0.5943609377704335, 0.5943609377704335, 1.0, 1.0, 1.0, 0.5943609377704335, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5943609377704335, 1.0, 1.0, 1.0, 0.5943609377704335, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5943609377704335, 0.5943609377704335, 1.0, 1.0, 1.0, 0.5943609377704335, 1.0, 1.0, 0.5943609377704335, 1.0, 1.0, 1.0, 1.0, 0.25, 0.25, 0.25, 1.0, 1.0, 1.0, 0.5943609377704335, 1.0, 1.0, 1.0, 1.0, 0.25, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5943609377704335, 1.0, 1.0, 1.0, 0.75, 0.75, 0.5943609377704335, 1.0, 0.5943609377704335, 1.0, 1.0, 1.0, 0.5, 0.5, 0.5]


In [96]:
#computeEntropy
for rf_ in families:
    print(rf_)
    errorsEntropy=[]
    try:
        families.get(rf_)['entropy'] = alignmentToShannon(families.get(rf_).get('gapless',None) )
    except AssertionError:
        errorsEntropy.append(rf_)

RF00001
RF00002
RF00003
RF00004
RF00005
RF00006
RF00007
RF00008
RF00009
RF00010
RF00011
RF00012
RF00013
RF00014
RF00015
RF00016
RF00017
RF00018
RF00019
RF00020
RF00021
RF00022
RF00023
RF00024
RF00025
RF00026
RF00027
RF00028
RF00029
RF00030
RF00031
RF00032
RF00033
RF00034
RF00035
RF00036
RF00037
RF00038
RF00039
RF00040
RF00041
RF00042
RF00043
RF00044
RF00045
RF00046
RF00047
RF00048
RF00049
RF00050
RF00051
RF00052
RF00053
RF00054
RF00055
RF00056
RF00057
RF00058
RF00059
RF00060
RF00061
RF00062
RF00063
RF00064
RF00065
RF00066
RF00067
RF00068
RF00069
RF00070
RF00071
RF00072
RF00073
RF00074
RF00075
RF00076
RF00077
RF00078
RF00079
RF00080
RF00081
RF00082
RF00083
RF00084
RF00085
RF00086
RF00087
RF00088
RF00089
RF00090
RF00091
RF00092
RF00093
RF00094
RF00095
RF00096
RF00097
RF00099
RF00100
RF00101
RF00102
RF00103
RF00104
RF00105
RF00106
RF00107
RF00108
RF00109
RF00110
RF00111
RF00112
RF00113
RF00114
RF00115
RF00116
RF00117
RF00118
RF00119
RF00121
RF00122
RF00124
RF00125
RF00126
RF00127
RF00128


RF01271
RF01272
RF01273
RF01274
RF01275
RF01276
RF01277
RF01278
RF01279
RF01280
RF01281
RF01283
RF01284
RF01285
RF01286
RF01287
RF01288
RF01289
RF01290
RF01291
RF01292
RF01293
RF01294
RF01295
RF01296
RF01297
RF01298
RF01299
RF01300
RF01301
RF01302
RF01303
RF01304
RF01305
RF01306
RF01307
RF01308
RF01309
RF01310
RF01312
RF01313
RF01314
RF01315
RF01316
RF01317
RF01318
RF01319
RF01320
RF01321
RF01322
RF01323
RF01324
RF01325
RF01326
RF01327
RF01328
RF01329
RF01330
RF01331
RF01332
RF01333
RF01334
RF01335
RF01336
RF01337
RF01338
RF01339
RF01340
RF01341
RF01342
RF01343
RF01344
RF01345
RF01346
RF01347
RF01348
RF01349
RF01350
RF01351
RF01352
RF01353
RF01354
RF01355
RF01356
RF01357
RF01358
RF01359
RF01360
RF01361
RF01362
RF01363
RF01364
RF01365
RF01366
RF01367
RF01368
RF01369
RF01370
RF01371
RF01373
RF01374
RF01375
RF01376
RF01377
RF01378
RF01379
RF01380
RF01381
RF01382
RF01383
RF01384
RF01385
RF01386
RF01387
RF01388
RF01389
RF01390
RF01391
RF01392
RF01393
RF01394
RF01395
RF01396
RF01397
RF01398


RF02423
RF02424
RF02425
RF02426
RF02427
RF02428
RF02429
RF02430
RF02431
RF02432
RF02433
RF02434
RF02435
RF02436
RF02437
RF02438
RF02439
RF02440
RF02441
RF02442
RF02443
RF02444
RF02445
RF02446
RF02447
RF02448
RF02449
RF02450
RF02451
RF02452
RF02453
RF02454
RF02455
RF02456
RF02457
RF02458
RF02459
RF02460
RF02461
RF02462
RF02463
RF02464
RF02465
RF02466
RF02467
RF02468
RF02469
RF02470
RF02471
RF02472
RF02473
RF02474
RF02475
RF02476
RF02477
RF02478
RF02479
RF02480
RF02481
RF02482
RF02483
RF02484
RF02485
RF02486
RF02487
RF02488
RF02489
RF02490
RF02491
RF02492
RF02493
RF02494
RF02495
RF02496
RF02497
RF02498
RF02499
RF02500
RF02501
RF02502
RF02503
RF02504
RF02505
RF02506
RF02507
RF02508
RF02509
RF02510
RF02511
RF02512
RF02513
RF02514
RF02515
RF02516
RF02517
RF02518
RF02519
RF02520
RF02521
RF02522
RF02523
RF02524
RF02525
RF02526
RF02527
RF02528
RF02529
RF02530
RF02531
RF02532
RF02533
RF02534
RF02535
RF02536
RF02537
RF02538
RF02539
RF02540
RF02541
RF02542
RF02543
RF02544
RF02545
RF02546
RF02547


In [97]:
families['RF00021']

{'SS_cons': '::::::::<<<<-<<<<<..-<<<<<<.<<<<<<___>>>>>>---->>>>>>...>>>>>-...>>..>>,,,,,,,,......,,<<<<<<_....>>>>>><<<<<<<<<<.__._._...>>>>>>>>>>::::::::',
 'align': ['AGUUAUUGGCGUAGGGUA--CAGAGGU-AAGAUGUUCUAUCUUUCAGACCUUU---UACUUC---AC--GUAAUCGGAU------UUAGCUGAAUA--UUAGCUGCCCCAGUCA-AU-U-U---UGACUGGGGCAUUUUUUU',
  'AGUUAGUCGCGUAGGGUA--CAGAGGU-AAGAUGUUCUAUCUUUCAGACCUUU---UACUUC---AC--GUAAUCGGAU------UUGGCUGAAUAUUUUAGCCGCCCCAGUCA-GU-A-A---UGACUGGGGCGUUUUUUA',
  'AGUUAUAGACGUAGGGUA--CAGAGGU-AAGAUGUUCUAUCUUUCAGACCUUU---UACUUC---AC--GUAAUCGGAU------UUGGCUGUAUA--UUAGCCGCCCCAGUCA-UU-UAU---UGACUGGGGCGUUUUUUG',
  'AUUAGGACGCGUAGGGUA--CAGAGGU-AAGAUGUUCUAUCUUUCAGACCUUU---UGUUUC---AC--GUUAUUGGAU------UAGGCUGAU----UCAGCCGCCCCAGUCA-GUAU-U---UGACUGGGGCGUUUUUUC',
  'AGUUAUCGGCGUAGGGUACACAGAGGU-AAGAUGUUCUAUCUUUCAGGCCUUU---UACUUC---AC--GUAAUCGGAC------UUGGCUAAGUA--UUAGCUGCCCCAGUCA-UU-U-A--AUGACUGGGGCGUUUUUUA',
  'AUUAAUACUCGUAGGGUA--CAGAGGU-AAGAUGUUCUAUCUUUCAGACCUUU---UGUUUC---AC--GUUAUUGGAU------UAGGC

In [103]:
def printEntropy(family_dict, fname= "test"):
    """given a dict d[rf]['entropy'] print on file"""
    o = open(fname,'w')
    for rf_ in family_dict:
        line = rf_ + "\t" + "\t".join([str(x) for x in family_dict[rf_]['entropy']])+"\n"
        o.write(line)
        
    o.close()
    
printEntropy(families, "../entropy/entropy.tsv")

In [ ]:
##plot part